# Análise comparativa de modelos

In [11]:
# Bibliotecas
#Bibliotecas 
from IPython.display import display, Markdown, HTML
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LinearRegression

## 1. Obtenção de dados

In [12]:
#Dicionário de dados
df = pd.read_csv("../data/raw/penguins.csv")
df_dict = pd.read_csv("../data/external/dictionary.csv")
df_dict

,variavel,descricao,tipo,subtipo
0,species,"Espécie do pinguim (Adelie, Chinstrap, Gentoo)",qualitativa,nominal
1,island,"Ilha onde o pinguim foi encontrado (Biscoe, Dr...",qualitativa,nominal
2,bill_length_mm,Comprimento do bico em milímetros,quantitativa,contínua
3,bill_depth_mm,Profundidade do bico em milímetros,quantitativa,contínua
4,flipper_length_mm,Comprimento da nadadeira em milímetros,quantitativa,contínua
5,body_mass_g,Massa corporal em gramas,quantitativa,contínua
6,sex,Sexo do pinguim (Masculino ou Feminino),qualitativa,nominal
7,year,Ano em que os dados foram coletados,quantitativa,discreta


# 2. Preparação de dados 

In [13]:
#Variável-alvo 
target_variable = "species"

#Variável Nominais
nominal_variables=(
    df_dict
    .query("subtipo == 'Nominal' and variavel != @target_variable ")
    .variavel
    .to_list()
)

#Variável Contínuas
continuous_variables=(
    df_dict
    .query("subtipo == 'Contínua' and variavel != @target_variable ")
    .variavel
    .to_list()
)

#Variável discreta
discrete_variables=(
    df_dict
    .query("subtipo == 'Discreta' and variavel != @target_variable ")
    .variavel
    .to_list()
)

X = df.drop(columns=[target_variable],axis=1)
y = df[target_variable]

# 2. Pipeline de Pré-processamento de dados

In [14]:
#Pipeline para variáveis nominais 
nominal_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='most_frequent')),
    ("encoding", OneHotEncoder(sparse_output=False)),
])
#Pipeline para variáveis contínuas
continuous_preprocessor = Pipeline(steps=[
    ("missing", SimpleImputer(strategy='mean')),
    ("normalization", StandardScaler()),
])

#Pipeline para variáveis discreta
discrete_preprocessor= Pipeline(steps=[
    ("missing", KNNImputer()),
    ("normalization", StandardScaler()),
])

In [15]:
## Criação do ColumnTransformer para processar cada tipo de variável
preprocessor= ColumnTransformer([
    ("nominal", nominal_preprocessor, nominal_variables),
    ("continuous", continuous_preprocessor, continuous_variables),
    ("discrete", discrete_preprocessor, discrete_variables),
    
])